In [ ]:
import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Lasso


def print_comparison(image, dct_coeffs, reconstructed_image, output_name, gan_image):
    dct_orig = np.log(np.abs(cv2.dct(np.float32(image))) + 1)
    dct_mod = np.log(np.abs(dct_coeffs) + 1)

    plt.figure(figsize=(16, 8))
    plt.subplot(2, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title(f'Original GAN Image {gan_image}')
    
    plt.subplot(2, 2, 2)
    plt.imshow(dct_orig, cmap='hot')
    plt.title('DCT original')
    plt.colorbar()
    
    
    plt.subplot(2, 2, 3)
    plt.imshow(reconstructed_image, cmap='gray')
    plt.title('After Fingerprint Removal')
    
    plt.subplot(2, 2, 4)
    plt.imshow(dct_mod, cmap='hot')
    plt.title('DCT of Detected Fingerprint')
    plt.colorbar()
    
    plt.savefig(output_name)
    plt.show()

## Frequency Bars Attack

In [ ]:
image = cv2.imread('../../data/PLUS/val/spoofed_synthethic_drit_003/026-PLUS-FV3-Laser_PALMAR_002_01_04_02.png', cv2.IMREAD_GRAYSCALE)
filename_comparison = glob.glob('../../data/PLUS/val/spoofed_synthethic_drit_003/026-PLUS-FV3-Laser_PALMAR_002_01_04_02.png')[0]
dataset = filename_comparison.split("/")[3]
attack = filename_comparison.split("/")[5].split("_")[-2:]
attack = "_".join(attack)
filename_comparison = filename_comparison.split("/")[-1].replace(".png", "")
filename_comparison = "_".join([dataset, attack, filename_comparison])

###### Start ATTACK ######
# Apply 2D DCT
dct_result = cv2.dct(np.float32(image))
fingerprint = dct_result.copy()
#set the last s rows and columns to zero
s=60
dct_result[-s:,:] = 0
dct_result[:,-s:] = 0
fingerprint -= dct_result
reconstructed_image = cv2.idct(dct_result)
###### End ATTACK ######

print_comparison(image, fingerprint, reconstructed_image, "../../plots/GAN_removal/attack_bars_PLUS_drit_003.png", filename_comparison)

## Mean Spectrum Attack

In [ ]:
# read filenames in a specified directory
filenames_genuine = glob.glob('../../data/PLUS/train/genuine/*.png')
filenames_cycle003 = glob.glob('../../data/PLUS/train/spoofed_synthethic_drit_003/*.png')

#create mean of dct coeffiecents of filenames_genuine
mean_dct_genuine = np.zeros(cv2.imread(filenames_genuine[0], cv2.IMREAD_GRAYSCALE).shape)
for filename in filenames_genuine:
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    mean_dct_genuine += cv2.dct(np.float32(image))
    
mean_dct_genuine /= len(filenames_genuine)

#create mean of dct coeffiecents of filenames_cycle003
mean_dct_cycle003 = np.zeros(cv2.imread(filenames_cycle003[0], cv2.IMREAD_GRAYSCALE).shape)
for filename in filenames_cycle003:
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    mean_dct_cycle003 += cv2.dct(np.float32(image))
    
mean_dct_cycle003 /= len(filenames_cycle003)

# Needs to be repeated for each gan class and applied on corresponding val images!!!!
fingerprint_mean = mean_dct_cycle003 - mean_dct_genuine


###### Start ATTACK ######
s=0.2
filename_cycle003_val = glob.glob('../../data/PLUS/val/spoofed_synthethic_drit_003/026-PLUS-FV3-Laser_PALMAR_002_01_04_02.png')[0]
image = cv2.imread(filename_cycle003_val, cv2.IMREAD_GRAYSCALE)
dct_result = cv2.dct(np.float32(image))
modified_coeffs = dct_result - (s*fingerprint_mean)
#store the new image in removal folder
reconstructed_image = cv2.idct(modified_coeffs)
print_comparison(image, fingerprint_mean, reconstructed_image, "../../plots/GAN_removal/attack_mean_PLUS_drit_003.png", filename_comparison)
###### End ATTACK ######


## Frequency Peaks Attack

In [ ]:
# read filenames in a specified directory
filenames_genuine = glob.glob('../../data/PLUS/train/genuine/*.png')
filenames_cycle003 = glob.glob('../../data/PLUS/train/spoofed_synthethic_drit_003/*.png')

#create mean of dct coeffiecents of filenames_genuine
mean_dct_genuine = np.zeros(cv2.imread(filenames_genuine[0], cv2.IMREAD_GRAYSCALE).shape)
for filename in filenames_genuine:
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    mean_dct_genuine += np.log(np.abs(cv2.dct(np.float32(image))))
    
mean_dct_genuine /= len(filenames_genuine)

#create mean of dct coeffiecents of filenames_cycle003
mean_dct_cycle003 = np.zeros(cv2.imread(filenames_cycle003[0], cv2.IMREAD_GRAYSCALE).shape)
for filename in filenames_cycle003:
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    mean_dct_cycle003 += np.log(np.abs(cv2.dct(np.float32(image))))
    
mean_dct_cycle003 /= len(filenames_cycle003)

# Needs to be repeated for each gan class and applied on corresponding val images!!!!
fingerprint_peak = np.exp(mean_dct_cycle003 - mean_dct_genuine)

###### Start ATTACK ######
# scale F_p to [0,1]
fingerprint_peak = (fingerprint_peak - np.min(fingerprint_peak)) / (np.max(fingerprint_peak) - np.min(fingerprint_peak))
t=0.1
#set values under threshold t to zero
fingerprint_peak[fingerprint_peak < t] = 0
s=100
fingerprint_peak *= s
#scale F_p to [0,1]
fingerprint_peak = (fingerprint_peak - np.min(fingerprint_peak)) / (np.max(fingerprint_peak) - np.min(fingerprint_peak))

filenames_cycle003_val = glob.glob('../../data/PLUS/val/spoofed_synthethic_drit_003/026-PLUS-FV3-Laser_PALMAR_002_01_04_02.png')[0]
image = cv2.imread(filenames_cycle003_val, cv2.IMREAD_GRAYSCALE)
dct_result = cv2.dct(np.float32(image))
modified_coeffs = dct_result * (1-fingerprint_peak)
#store the new image in removal folder
reconstructed_image = cv2.idct(modified_coeffs)
print_comparison(image, fingerprint_peak, reconstructed_image, "../../plots/GAN_removal/attack_peaks_PLUS_drit_003.png", filename_comparison)
###### End ATTACK ######

## Regression Weights Attack

In [ ]:
# train a lasso model on genuine dct coeffs
filenames_genuine = glob.glob('../../data/PLUS/train/genuine/*.png')
filenames_cycle003 = glob.glob('../../data/PLUS/train/spoofed_synthethic_drit_003/*.png')

#create mean of dct coeffiecents of filenames_genuine
lasso_model = Lasso(alpha=0.001 ,max_iter=10, tol=0.1) 
for filename in filenames_genuine[:1]:
    for filename_cycle003 in filenames_cycle003:
        print(f"step: {filename} {filename_cycle003}")
        image_gen = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        image_gan = cv2.imread(filename_cycle003, cv2.IMREAD_GRAYSCALE)
        dct_result_gen = cv2.dct(np.float32(image_gen))
        dct_result_gan = cv2.dct(np.float32(image_gan))
        lasso_model.fit(dct_result_gan, dct_result_gen)

###### Start ATTACK ######        
F_r = np.resize(lasso_model.coef_, image_gan.shape)
# scale F_r to [-1,1]
F_r = 2 * (F_r - np.min(F_r)) / (np.max(F_r) - np.min(F_r)) - 1
filenames_cycle003_val = glob.glob('../../data/PLUS/val/spoofed_synthethic_drit_003/026-PLUS-FV3-Laser_PALMAR_002_01_04_02.png')[0]
image = cv2.imread(filenames_cycle003_val, cv2.IMREAD_GRAYSCALE)
dct_result = cv2.dct(np.float32(image))

modified_coeffs = (1-F_r) * dct_result
#store the new image in removal folder
reconstructed_image = cv2.idct(modified_coeffs)
print_comparison(image, F_r, reconstructed_image, "../../plots/GAN_removal/attack_regression_weights_PLUS_drit_003.png", filename_comparison)
###### End ATTACK ######
